In [1]:
import os
import datetime
import pandas as pd
import numpy as np
os.chdir('../')
from rsutils import data_utils as du, tagging_utils as tu
os.chdir('C:/Users/ryanm_fa0xwnc/Documents/Github/sif_project')
from siftools import sif_utils as su

In [2]:
# Get current tickers in US equities market
metadata = su.get_metadata()
metadata['lastpricedate'] = pd.to_datetime(metadata['lastpricedate'])
metadata = metadata[metadata['lastpricedate'] >= datetime.datetime.now() - datetime.timedelta(30)]
tickers = metadata['ticker'].unique().tolist()

In [3]:
comments = pd.read_csv('C:/Users/ryanm_fa0xwnc/PyCharmProjects/nlp_research/comments_stream.csv')  # Read comments data
comments = comments[['id', 'body', 'score', 'created_utc']]  # Select relevant columns
comments = comments[comments['body'].str.len() > 21]  # Filter comments less than 20 characters
comments = tu.apply_keywords_tags(comments, tickers, 'body', True)  # Extract tickers from comments
comments = comments[comments['body_keywords'].astype(bool)]  # Remove comments without any mention of ticker

In [4]:
# Get negative or positive sentiment label and convert from [0, 1] to [-1, 1]
comments = tu.apply_sentiment_tags(
    comments,
    'distilbert-base-uncased-finetuned-sst-2-english',
    'body'
)
comments['body_sentiment'] = (comments['body_sentiment'] * 2) - 1
# Use comment score to weight the sentiment
comments['sentiment_score'] = comments['body_sentiment'] * comments['score']

In [5]:
def multi_hot_encode(labels):
    unique_labels = np.unique(np.concatenate(labels, axis=0))
    def get_encoding(label, unique):
        return pd.Series(unique).isin(label).astype(int)
    encoded = pd.Series(labels).apply(lambda label: get_encoding(label, unique_labels))
    encoded.columns = unique_labels
    return encoded

In [6]:
encoded = multi_hot_encode(comments['body_keywords'].values)
ticker_scores = encoded.T * comments['sentiment_score'].values
ticker_scores.sum(axis=1).sort_values(ascending=False)

AGG     1
BNDW    1
GE      1
SHV     1
ITOT    1
IVV     1
VXUS    0
QQQ     0
NAV     0
E      -1
GOOD   -1
GM     -1
DD     -1
DIS    -1
HY     -1
CEO    -1
CEF    -1
AR     -1
AAPL   -1
H      -1
IMO    -1
VT     -1
NVDA   -1
TSLA   -1
TMF    -1
TLT    -1
INTC   -1
SCHZ   -1
LOVE   -1
KO     -1
TD     -2
ITM    -2
AMZN   -2
TQQQ   -2
AMD    -5
IPO    -5
A      -5
G      -6
dtype: int64